# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from ipywidgets.embed import embed_minimal_html


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cleaned_weather_df=pd.read_csv('../output_data/cities.csv')
cleaned_weather_df.drop(axis=1, labels='Unnamed: 0', inplace=True)
cleaned_weather_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,jamestown,US,42.10,-79.24,77.00,60,40,6.93,1596143156
1,lhokseumawe,ID,5.18,97.15,77.81,82,100,3.42,1596143156
2,avarua,CK,-21.21,-159.78,69.80,64,100,5.82,1596143156
3,albany,US,42.60,-73.97,80.01,51,93,6.44,1596143156
4,sumbe,AO,-11.21,13.84,69.51,88,1,9.42,1596143047
...,...,...,...,...,...,...,...,...,...
568,naliya,IN,23.27,68.83,85.05,77,100,8.39,1596143218
569,nha trang,VN,12.25,109.18,82.40,88,75,3.36,1596143218
570,louis trichardt,ZA,-23.04,29.90,55.96,39,0,4.07,1596143218
571,durban,ZA,-29.86,31.03,68.00,77,75,5.82,1596142998


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Create df for lat long locations
locations = cleaned_weather_df[["Latitude", "Longitude"]].astype(float)

#Create series for humidity
humidity=cleaned_weather_df['Humidity'].astype(float)

# Create and add humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)

#Display heatmap
fig
embed_minimal_html('../output_data/Heatmap.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
ideal_weather_df=cleaned_weather_df

#A max temperature lower than 80 degrees but higher than 70.
ideal_weather_df=ideal_weather_df[ideal_weather_df['Max Temp']>70]
ideal_weather_df=ideal_weather_df[ideal_weather_df['Max Temp']<80]

#Wind speed less than 10 mph.
ideal_weather_df=ideal_weather_df[ideal_weather_df['Wind Speed']<10]

#Cloudiness less than 5.
ideal_weather_df=ideal_weather_df[ideal_weather_df['Cloudiness']<5]

#Display ideal weather df
ideal_weather_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
15,takoradi,GH,4.88,-1.76,74.53,86,0,8.72,1596142860
53,dawlatabad,AF,36.41,64.91,76.75,19,0,3.74,1596143161
74,mayor pablo lagerenza,PY,-19.93,-60.77,73.71,34,2,4.36,1596143163
141,hecun,CN,36.53,114.11,74.64,86,0,0.58,1596143171
150,sremska mitrovica,RS,44.98,19.61,77.00,72,0,2.93,1596143018
157,bad orb,DE,50.23,9.35,72.00,51,2,3.20,1596143173
190,haines junction,CA,60.75,-137.51,74.52,40,0,8.57,1596143176
199,fethiye,TR,36.62,29.12,78.80,83,0,2.24,1596143177
210,vila,PT,42.03,-8.16,71.01,74,0,5.01,1596143178
271,demba,CD,-5.51,22.27,72.12,29,0,2.57,1596143184


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=ideal_weather_df

In [6]:
url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
params={
    'radius':5000,
    'type': 'lodging',
    'key': g_key
}

In [7]:
hotel_name_list=[]
for index, row in hotel_df.iterrows():
    try:
        response=requests.get(url+str(row['Latitude'])+','+str(row['Longitude']),params=params).json()
        hotel_name_list.append(response['results'][0]['name'])
    except IndexError:
        hotel_name_list.append(None)
hotel_df['Hotel Name']=hotel_name_list
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
15,takoradi,GH,4.88,-1.76,74.53,86,0,8.72,1596142860,Raybow International Hotel
53,dawlatabad,AF,36.41,64.91,76.75,19,0,3.74,1596143161,WASIM ASGAR KHAN
74,mayor pablo lagerenza,PY,-19.93,-60.77,73.71,34,2,4.36,1596143163,None
141,hecun,CN,36.53,114.11,74.64,86,0,0.58,1596143171,Yijia Hotel
150,sremska mitrovica,RS,44.98,19.61,77.00,72,0,2.93,1596143018,Apartment Velago
157,bad orb,DE,50.23,9.35,72.00,51,2,3.20,1596143173,Helvetia
190,haines junction,CA,60.75,-137.51,74.52,40,0,8.57,1596143176,Parkside Inn
199,fethiye,TR,36.62,29.12,78.80,83,0,2.24,1596143177,Club & Hotel Letoonia
210,vila,PT,42.03,-8.16,71.01,74,0,5.01,1596143178,Hotel Castrum Villae
271,demba,CD,-5.51,22.27,72.12,29,0,2.57,1596143184,None


In [8]:
hotel_df.dropna(inplace=True)
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
15,takoradi,GH,4.88,-1.76,74.53,86,0,8.72,1596142860,Raybow International Hotel
53,dawlatabad,AF,36.41,64.91,76.75,19,0,3.74,1596143161,WASIM ASGAR KHAN
141,hecun,CN,36.53,114.11,74.64,86,0,0.58,1596143171,Yijia Hotel
150,sremska mitrovica,RS,44.98,19.61,77.00,72,0,2.93,1596143018,Apartment Velago
157,bad orb,DE,50.23,9.35,72.00,51,2,3.20,1596143173,Helvetia
190,haines junction,CA,60.75,-137.51,74.52,40,0,8.57,1596143176,Parkside Inn
199,fethiye,TR,36.62,29.12,78.80,83,0,2.24,1596143177,Club & Hotel Letoonia
210,vila,PT,42.03,-8.16,71.01,74,0,5.01,1596143178,Hotel Castrum Villae
284,shingu,JP,33.73,135.98,74.01,90,0,3.51,1596143186,Shingu UI Hotel
418,solhan,TR,38.96,41.03,70.41,38,0,4.97,1596143200,Grand Konak Otel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
\n
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Create df for lat long locations
locations = hotel_df[["Latitude", "Longitude"]].astype(float)

#Create series for humidity
humidity=hotel_df['Humidity'].astype(float)

# Create and add humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)


# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)
#markers = gmaps.marker_layer(locations=locations, info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_df['Hotel Name']])
fig.add_layer(markers)

# Display figure
fig
embed_minimal_html('../output_data/Ideal_Hotels.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))